# Threshold Joye-Libert Secure Aggregation

In [1]:
from ftsa.protocols.buildingblocks.JoyeLibert import TJLS
import random

t = 412
threshold = 7
nclients = 10
drops = 3
keysize = 2048 
inputsize = 16
dimension = 100

users = list(range(1, nclients+1))
alive = list(range(1, nclients-drops+1))
dropped = list(range(nclients-drops+1,nclients+1))


TJL = TJLS(nclients, threshold)

print("users: ", users)
print("alive: ", alive)
print("dropped: ", dropped)

users:  [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
alive:  [1, 2, 3, 4, 5, 6, 7]
dropped:  [8, 9, 10]


### Generate JL keys (**Setup**)

In [2]:
# generate keys
pp, skey, ukey = TJL.Setup(keysize)

print(" - Public Parameters:", pp)
print(" - Aggregator Key:", skey)
print(" - User Keys:", ukey)

 - Public Parameters: <PublicParam (N=11537...77537, H(x)=0x1938d02)>
 - Aggregator Key: <ServerKey -0x1ddb0b4>
 - User Keys: {0: <UserKey 0x1df6666e>, 1: <UserKey 0xc502babd>, 2: <UserKey 0x5dc925e0>, 3: <UserKey 0x175f1987>, 4: <UserKey 0x73dc48f0>, 5: <UserKey 0x17440867>, 6: <UserKey 0x1bce9351>, 7: <UserKey 0xfe0d5083>, 8: <UserKey 0x1e9911f0>, 9: <UserKey 0xd8e8603a>}


### Share the JL user key with all other users (**SKShare**)


In [3]:
# share secret key of each user
shares = {}
for u in users: 
    shares[u] = TJL.SKShare(ukey[u-1],threshold, users)

# distribute the shares
from collections import defaultdict
usersshares = defaultdict(dict)
for v in users:
    for u in users: 
        usersshares[u][v] = shares[v][u-1]

CPU times: user 0 ns, sys: 2.64 ms, total: 2.64 ms
Wall time: 1.92 ms


### Protect inputs of alive users with the users' keys (**Protect**)

In [4]:
# generate $nclients random numbers
l =[]
for i in range(0,nclients):
    l.append(random.randint(0,2**16))

print("Clients Inputs: ", l )


# protect each number with a different client key
e=[]
for u in alive:
    e.append(TJL.Protect(pp, ukey[u-1], t, l[u-1]))

e

Clients Inputs:  [23606, 21316, 24226, 36192, 11696, 30082, 42691, 59692, 18397, 9939]


[<EncryptedNumber 10901...35235>,
 <EncryptedNumber 11485...00440>,
 <EncryptedNumber 12617...78525>,
 <EncryptedNumber 11916...64923>,
 <EncryptedNumber 77959...47247>,
 <EncryptedNumber 10686...49772>,
 <EncryptedNumber 74869...53401>]

### Protect the zero-value with the key shares of the dropped users  (**ShareProtect**)

In [5]:
if dropped != []:
    yzeroshare ={}
    for u in alive:
        dropped_users_shares = []
        for v in dropped:
            dropped_users_shares.append(usersshares[u][v])
        yzeroshare[u] = TJL.ShareProtect(pp, dropped_users_shares,t)


### Combine shares of the protected zero-value (**ShareCombine**)

In [6]:
if dropped != []:
    yzero = TJL.ShareCombine(pp, list(yzeroshare.values()), threshold)
    print("yzero:", yzero)
else:
    yzero = None

yzero: <EncryptedNumber 54941...91890>


### Aggregate all user protected inputs and the protected zero-value of dropped users (**Agg**)

In [7]:
s = TJL.Agg(pp, skey, t, e, yzero)


print("Sum=", s )
# check if the result is correct
print("Verify=", s == sum(l[:nclients-drops]))

Sum= 189809
Verify= True
